### Update bib records via Alma API

update several bib records through the alma api, replace the url in field 856$u and create field 024 with ark id.

The script reads the input file named 'SoSaGraphics_ARK_MMSIDs.json' (generated by other script) and loops through the list of ID-URL pairs. It downloads the bib record in marcxml, finds the 856 field, updates the url and creates an identifier field (024) and uploads the new bib record.
This script assumes there is only one field 856 $$u.

To run this script, add the following:

Write your Alma API key in a file named config.py in the same folder as your script. The file should look like this:

(This file is under .gitignore, the API keys are not publicly shared)


In [44]:
# api_key for SLSP NZ Sandbox or production

api_key_nz = "NZ_API_KEY"



This script assumes that field 856 contains the old links to the Sosa graphics (edoc.zhbluzern.ch) and can be overwritten (which is true for these ca. 2300 titles, but may have to be checked for another purpose). 

In [8]:
import requests
import urllib.request
import json
import xml.etree.ElementTree as ET
import re
import config

# Set your Alma API key, base API endpoint, input file name
api_key = config.api_key_nz
base_url = 'https://api-eu.hosted.exlibrisgroup.com/almaws/v1'
filename = 'SoSaGraphics_ARK_MMSIDs_20230310.json'
#filename = 'sosatest01.json'
#filename = 'sosatest10.json'

# Opening JSON file and return JSON object as a dictionary
f = open(filename)
id_url_list = json.load(f)
# debug: print(id_url_list)

# Loop through the list and update each bib record
# add debug nr. if loop aborts for some reason: for i, bib in enumerate(id_url_list[499:]):
for i, bib in enumerate(id_url_list):    
    print(i)
    nzid = bib['networkID']
    #print(nzid)
    ark = "https://n2t.net/"+bib['ARK']
    # Construct the API URL to get the bib record in MARCXML format
    bib_url = f'{base_url}/bibs/{nzid}?view=full&expand=None&apikey={api_key}'

    # Send the request to get the bib record
    response = requests.get(bib_url)
    
    # create backup
    backupfile = "backup-xml/" + nzid + ".xml"
    with open(backupfile, "w", encoding = "utf-8") as f:
        f.write(response.text)

    # Parse the MARCXML data from the response content
    marcxml = ET.fromstring(response.content)  
    
    # Find the 856 field you want to replace
    f856 = marcxml.findall('.//datafield[@tag="856"]')
    if len(f856) > 0:
        for field in marcxml.findall('.//datafield[@tag="856"]'):
            # Find the subfield with the URL
            url_subfield = field.find('.//subfield[@code="u"]')
            if url_subfield is not None:
            # Replace the URL with the new value
                url_subfield.text = ark
                # debug: print(url_subfield.text)
    else:
        #create new 856 url with ark
        print("no 856 exists")
        record = marcxml.find("record")
        new_856 = ET.Element("datafield", attrib={"ind1": "4", "ind2": " ", "tag": "856"})
        new_8563 = ET.Element("subfield", attrib={"code": "3"})
        new_8563.text = "Bild"
        new_856u = ET.Element("subfield", attrib={"code": "u"})
        new_856u.text = ark
        new_856.append(new_8563)
        new_856.append(new_856u)
        record.append(new_856)

    # Create a new 024 field with subfields a and 2
    # First check if there is a Field 024 matching the given ARK to avoid duplicate entries
    marc_024 = marcxml.findall(".//datafield[@tag='024']/subfield[@code='a']")
    match_024 = False
    for field in marc_024:
        if field.text == bib["ARK"]:
            print("ark match")
            match_024 = True
    if match_024 == False:
        record = marcxml.find("record")
        new_024 = ET.Element("datafield", attrib={"ind1": "7", "ind2": " ", "tag": "024"})
        new_024a = ET.Element("subfield", attrib={"code": "a"})
        new_024a.text = bib['ARK']
        new_0242 = ET.Element("subfield", attrib={"code": "2"})
        new_0242.text = 'ark'
        new_024.append(new_024a)
        new_024.append(new_0242)
        record.append(new_024)
        #debug: print(ET.tostring(new_024))
    
    # Create a new license field 
    # 540 $aPublic Domain Mark 1.0$uhttp://creativecommons.org/publicdomain/mark/1.0/$2cc
    marc_540 = marcxml.findall(".//datafield[@tag='540']/subfield[@code='a']")
    match_540 = False
    for field in marc_540:
        if field.text == "Public Domain Mark 1.0":
            print("license match")
            match_540 = True
    if match_540 == False:
        record = marcxml.find("record")
        new_540 = ET.Element("datafield", attrib={"ind1": " ", "ind2": " ", "tag": "540"})
        new_540a = ET.Element("subfield", attrib={"code": "a"})
        new_540a.text = 'Public Domain Mark 1.0'
        new_540u = ET.Element("subfield", attrib={"code": "u"})
        new_540u.text = 'https://creativecommons.org/publicdomain/mark/1.0/'        
        new_5402 = ET.Element("subfield", attrib={"code": "2"})
        new_5402.text = 'cc'
        new_540.append(new_540a)
        new_540.append(new_540u)
        new_540.append(new_5402)
        record.append(new_540)
        #debug: print(ET.tostring(new_540))

    # Create the restriction on access field
    # 506 $$a Open Access $$f Unrestricted online access $$q DE-14 $$u http://purl.org/coar/access_right/c_abf2 $$2 star
    marc_506 = marcxml.findall(".//datafield[@tag='506']/subfield[@code='a']")
    match_506 = False
    for field in marc_506:
        if field.text == "Open Access":
            print("restriction on access match")
            match_506 = True
    if match_506 == False:
        record = marcxml.find("record")
        new_506 = ET.Element("datafield", attrib={"ind1": " ", "ind2": " ", "tag": "506"})
        new_506a = ET.Element("subfield", attrib={"code": "a"})
        new_506a.text = 'Open Access'
        new_506f = ET.Element("subfield", attrib={"code": "f"})
        new_506f.text = 'Unrestricted online access'
        new_506u = ET.Element("subfield", attrib={"code": "u"})
        new_506u.text = 'http://purl.org/coar/access_right/c_abf2'        
        new_5062 = ET.Element("subfield", attrib={"code": "2"})
        new_5062.text = 'star'
        new_506.append(new_506a)
        new_506.append(new_506u)
        new_506.append(new_5062)
        record.append(new_506)
        #debug: print(ET.tostring(new_540))

    record = marcxml.find("record")
    record[:] = sorted(record, key=lambda field_or_contr: field_or_contr.get('tag', '000'))

    # Convert the updated MARCXML back to a string
    updated_marcxml = ET.tostring(marcxml, encoding='utf-8')

    # Construct the API URL to update the bib record
    update_url = f'{base_url}/bibs/{nzid}?apikey={api_key}'

    # Send the request to update the bib record with the updated MARCXML
    response = requests.put(update_url, data=updated_marcxml, headers={'Content-Type': 'application/xml'})

    # Print the response status code to verify the update was successful
    print(f'Bib record {nzid} updated with URL {ark} - Response code: {response.status_code}')
    #debug: break

0
ark match
license match
restriction on access match
Bib record 991159028839705501 updated with URL https://n2t.net/ark:/63274/zhb1dq2r - Response code: 200
1
ark match
license match
restriction on access match
Bib record 991050167169705501 updated with URL https://n2t.net/ark:/63274/zhb1jh6k - Response code: 200
2
ark match
license match
restriction on access match
Bib record 991049253629705501 updated with URL https://n2t.net/ark:/63274/zhb16t0t - Response code: 200
3
ark match
license match
restriction on access match
Bib record 991117838399705501 updated with URL https://n2t.net/ark:/63274/zhb1bh4d - Response code: 200
4
ark match
license match
restriction on access match
Bib record 991170996258505501 updated with URL https://n2t.net/ark:/63274/zhb1g96b - Response code: 200
5
ark match
license match
restriction on access match
Bib record 991170996252605501 updated with URL https://n2t.net/ark:/63274/zhb1m081 - Response code: 200
6
ark match
license match
restriction on access matc

Bib record 991160368559705501 updated with URL https://n2t.net/ark:/63274/zhb18371 - Response code: 200
73
Bib record 991118736059705501 updated with URL https://n2t.net/ark:/63274/zhb1ct0q - Response code: 200
74
Bib record 991159378179705501 updated with URL https://n2t.net/ark:/63274/zhb1hm2n - Response code: 200
75
Bib record 991015797299705501 updated with URL https://n2t.net/ark:/63274/zhb1n95s - Response code: 200
76
Bib record 991075676659705501 updated with URL https://n2t.net/ark:/63274/zhb1s09c - Response code: 200
77
Bib record 991064428029705501 updated with URL https://n2t.net/ark:/63274/zhb1wt1v - Response code: 200
78
Bib record 991090273609705501 updated with URL https://n2t.net/ark:/63274/zhb11h3j - Response code: 200
79
Bib record 991143000899705501 updated with URL https://n2t.net/ark:/63274/zhb1595g - Response code: 200
80
Bib record 991088023609705501 updated with URL https://n2t.net/ark:/63274/zhb19092 - Response code: 200
81
Bib record 991022355329705501 updated

Bib record 991056400809705501 updated with URL https://n2t.net/ark:/63274/zhb16m27 - Response code: 200
150
Bib record 991071646699705501 updated with URL https://n2t.net/ark:/63274/zhb1b95c - Response code: 200
151
Bib record 991028601869705501 updated with URL https://n2t.net/ark:/63274/zhb1g379 - Response code: 200
152
Bib record 991127261589705501 updated with URL https://n2t.net/ark:/63274/zhb1kt00 - Response code: 200
153
Bib record 991147687619705501 updated with URL https://n2t.net/ark:/63274/zhb1qh34 - Response code: 200
154
Bib record 991137785499705501 updated with URL https://n2t.net/ark:/63274/zhb1v952 - Response code: 200
155
Bib record 991066108329705501 updated with URL https://n2t.net/ark:/63274/zhb1009n - Response code: 200
156
Bib record 991160368479705501 updated with URL https://n2t.net/ark:/63274/zhb13t0p - Response code: 200
157
Bib record 991090365229705501 updated with URL https://n2t.net/ark:/63274/zhb17h48 - Response code: 200
158
Bib record 99113887340970550

Bib record 991150247009705501 updated with URL https://n2t.net/ark:/63274/zhb1195z - Response code: 200
226
Bib record 991035261829705501 updated with URL https://n2t.net/ark:/63274/zhb1537w - Response code: 200
227
Bib record 991130959509705501 updated with URL https://n2t.net/ark:/63274/zhb18s96 - Response code: 200
228
Bib record 991088015679705501 updated with URL https://n2t.net/ark:/63274/zhb1dm2h - Response code: 200
229
Bib record 991147388659705501 updated with URL https://n2t.net/ark:/63274/zhb1j946 - Response code: 200
230
Bib record 991016510409705501 updated with URL https://n2t.net/ark:/63274/zhb1p07b - Response code: 200
231
Bib record 991063621769705501 updated with URL https://n2t.net/ark:/63274/zhb1ss9w - Response code: 200
232
Bib record 991055680219705501 updated with URL https://n2t.net/ark:/63274/zhb1xh3d - Response code: 200
233
Bib record 991082000089705501 updated with URL https://n2t.net/ark:/63274/zhb1294w - Response code: 200
234
Bib record 99109089144970550

Bib record 991095766129705501 updated with URL https://n2t.net/ark:/63274/zhb1v361 - Response code: 200
302
Bib record 991127266169705501 updated with URL https://n2t.net/ark:/63274/zhb1zt05 - Response code: 200
303
Bib record 991066918959705501 updated with URL https://n2t.net/ark:/63274/zhb13m23 - Response code: 200
304
Bib record 991049593659705501 updated with URL https://n2t.net/ark:/63274/zhb1794s - Response code: 200
305
Bib record 991135824499705501 updated with URL https://n2t.net/ark:/63274/zhb1c08c - Response code: 200
306
Bib record 991039401329705501 updated with URL https://n2t.net/ark:/63274/zhb1gt0v - Response code: 200
307
Bib record 991054267779705501 updated with URL https://n2t.net/ark:/63274/zhb1mh2j - Response code: 200
308
Bib record 991033931539705501 updated with URL https://n2t.net/ark:/63274/zhb1r94g - Response code: 200
309
Bib record 991101695499705501 updated with URL https://n2t.net/ark:/63274/zhb1w082 - Response code: 200
310
Bib record 99105101406970550

...

Bib record 991155378959705501 updated with URL https://n2t.net/ark:/63274/zhb1vk5c - Response code: 200
1974
Bib record 991091429059705501 updated with URL https://n2t.net/ark:/63274/zhb1089z - Response code: 200
1975
Bib record 991058758439705501 updated with URL https://n2t.net/ark:/63274/zhb1430c - Response code: 200
1976
Bib record 991159732909705501 updated with URL https://n2t.net/ark:/63274/zhb17s4k - Response code: 200
1977
Bib record 991106882179705501 updated with URL https://n2t.net/ark:/63274/zhb1ck52 - Response code: 200
1978
Bib record 991035648929705501 updated with URL https://n2t.net/ark:/63274/zhb1h89n - Response code: 200
1979
Bib record 991142697509705501 updated with URL https://n2t.net/ark:/63274/zhb1n31h - Response code: 200
1980
Bib record 991007991609705501 updated with URL https://n2t.net/ark:/63274/zhb1rs3t - Response code: 200
1981
Bib record 991085347599705501 updated with URL https://n2t.net/ark:/63274/zhb1wk66 - Response code: 200
1982
Bib record 99111041

Bib record 991126261529705501 updated with URL https://n2t.net/ark:/63274/zhb1pc8b - Response code: 200
2050
Bib record 991070667719705501 updated with URL https://n2t.net/ark:/63274/zhb1t30z - Response code: 200
2051
Bib record 991070669309705501 updated with URL https://n2t.net/ark:/63274/zhb1xs45 - Response code: 200
2052
Bib record 991094692849705501 updated with URL https://n2t.net/ark:/63274/zhb12k5n - Response code: 200
2053
Bib record 991126261569705501 updated with URL https://n2t.net/ark:/63274/zhb16897 - Response code: 200
2054
Bib record 991142886959705501 updated with URL https://n2t.net/ark:/63274/zhb1b30n - Response code: 200
2055
Bib record 991110553709705501 updated with URL https://n2t.net/ark:/63274/zhb1fs4v - Response code: 200
2056
Bib record 991037991999705501 updated with URL https://n2t.net/ark:/63274/zhb1kk6s - Response code: 200
2057
Bib record 991001586449705501 updated with URL https://n2t.net/ark:/63274/zhb1q88g - Response code: 200
2058
Bib record 99116025

Bib record 991088781439705501 updated with URL https://n2t.net/ark:/63274/zhb1h30j - Response code: 200
2126
Bib record 991111566119705501 updated with URL https://n2t.net/ark:/63274/zhb1ms4r - Response code: 200
2127
Bib record 991160360879705501 updated with URL https://n2t.net/ark:/63274/zhb1rk57 - Response code: 200
2128
Bib record 991160360839705501 updated with URL https://n2t.net/ark:/63274/zhb1w89t - Response code: 200
2129
Bib record 991048893419705501 updated with URL https://n2t.net/ark:/63274/zhb11307 - Response code: 200
2130
Bib record 991039401389705501 updated with URL https://n2t.net/ark:/63274/zhb14s4f - Response code: 200
2131
Bib record 991006594639705501 updated with URL https://n2t.net/ark:/63274/zhb18k5x - Response code: 200
2132
Bib record 991088755949705501 updated with URL https://n2t.net/ark:/63274/zhb1d882 - Response code: 200
2133
Bib record 991080521239705501 updated with URL https://n2t.net/ark:/63274/zhb1j30x - Response code: 200
2134
Bib record 99100154

Bib record 991034591849705501 updated with URL https://n2t.net/ark:/63274/zhb19w2p - Response code: 200
2202
Bib record 991160370239705501 updated with URL https://n2t.net/ark:/63274/zhb1fk5t - Response code: 200
2203
Bib record 991091486219705501 updated with URL https://n2t.net/ark:/63274/zhb1k88z - Response code: 200
2204
Bib record 991139081949705501 updated with URL https://n2t.net/ark:/63274/zhb1q30t - Response code: 200
2205
Bib record 991000327489705501 updated with URL https://n2t.net/ark:/63274/zhb1ts3k - Response code: 200
2206
Bib record 991089774609705501 updated with URL https://n2t.net/ark:/63274/zhb1zk5h - Response code: 200
2207
Bib record 991127561299705501 updated with URL https://n2t.net/ark:/63274/zhb1388n - Response code: 200
2208
Bib record 991143002419705501 updated with URL https://n2t.net/ark:/63274/zhb1730h - Response code: 200
2209
Bib record 991018214549705501 updated with URL https://n2t.net/ark:/63274/zhb1bs38 - Response code: 200
2210
Bib record 99106673

Bib record 991055614659705501 updated with URL https://n2t.net/ark:/63274/zhb14k5d - Response code: 200
2278
Bib record 991072353529705501 updated with URL https://n2t.net/ark:/63274/zhb18c7b - Response code: 200
2279
Bib record 991035224719705501 updated with URL https://n2t.net/ark:/63274/zhb1d30d - Response code: 200
2280
Bib record 991057790429705501 updated with URL https://n2t.net/ark:/63274/zhb1hs35 - Response code: 200
2281
Bib record 991015791229705501 updated with URL https://n2t.net/ark:/63274/zhb1nk53 - Response code: 200
2282
Bib record 991000214159705501 updated with URL https://n2t.net/ark:/63274/zhb1s887 - Response code: 200
2283
Bib record 991019227789705501 updated with URL https://n2t.net/ark:/63274/zhb1x303 - Response code: 200
2284
Bib record 991089208189705501 updated with URL https://n2t.net/ark:/63274/zhb11s3v - Response code: 200
2285
Bib record 991161040219705501 updated with URL https://n2t.net/ark:/63274/zhb15k5s - Response code: 200
2286
Bib record 99111156